<a href="https://colab.research.google.com/github/ash12hub/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd;
import numpy as np;

In [195]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data", header=None);
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
yes = 1;
no = 0;
zero = 0;
df = df.replace(["?", "n", "y"], [np.NAN, no, yes])

In [197]:
df[[zero, 1]].head()

,0,1
0,republican,0.0
1,republican,0.0
2,democrat,NaN
3,democrat,0.0
4,democrat,1.0


In [0]:
# List for each issue
dfs = []
party = "Party";
vote = "Vote";

for i in range(1, 17):
  checkdf = df[[zero, i]].dropna()
  checkdf = checkdf.rename(columns={zero: party, i: vote})
  dfs.append(checkdf)

In [199]:
dfs[7].head()

,Party,Vote
0,republican,0.0
1,republican,0.0
2,democrat,0.0
3,democrat,0.0
4,democrat,0.0


In [205]:
#Democrats and Republicans for each issue
dfsDemocrat = []
dfsRepublican =  []

for df in dfs:
  democratList = df[df[party] == "democrat"]
  dfsDemocrat.append(democratList)
  republicanList = df[df[party] == "republican"]
  dfsRepublican.append(republicanList)

print(dfsDemocrat[0].shape, dfsRepublican[0].shape)

(258, 2) (165, 2)


In [206]:
print(dfsDemocrat[1].shape, dfsRepublican[1].shape)

(239, 2) (148, 2)


In [208]:
dfsRepublican[2][dfsRepublican[2][vote] == yes].head()

,Party,Vote
71,republican,1.0
73,republican,1.0
117,republican,1.0
166,republican,1.0
167,republican,1.0


In [0]:
# Hypothesis Testing Start
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel;

In [135]:
dfsDemocrat[0].describe()

,1
count,258.000000
mean,0.604651
std,0.489876
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [136]:
dfsRepublican[1].shape

(148, 2)

In [194]:
dfsDemocrat[0][vote].head()

2    ?
3    n
4    y
5    n
6    n
Name: Vote, dtype: object

In [191]:
print(dfsDemocrat[0][dfsDemocrat[0][vote] == yes].shape[0])

0


In [213]:
for i in range(1, 17):
  if(dfsDemocrat[i-1][dfsDemocrat[i-1][vote] == yes].shape[0] > dfsRepublican[i-1][dfsRepublican[i-1][vote] == yes].shape[0]):
    statistics, pvalue = ttest_ind(dfsDemocrat[i-1][vote], dfsRepublican[i-1][vote])
    print((i-1), 'statistics:', statistics, 'pvalue:', pvalue)
    print(pvalue < 0.01)

0 statistics: 9.205264294809222 pvalue: 1.613440327936998e-18
True
1 statistics: -0.08896538137868286 pvalue: 0.9291556823994811
False
2 statistics: 23.21277691701378 pvalue: 2.0703402795405602e-77
True
6 statistics: 12.526187929077842 pvalue: 8.521033017443427e-31
True
7 statistics: 18.052093200819733 pvalue: 2.824718413723432e-54
True
8 statistics: 16.43750326854299 pvalue: 5.030792653107883e-47
True
9 statistics: -1.7359117329695164 pvalue: 0.08330248490425282
False
10 statistics: 8.293603989407588 pvalue: 1.5759322301054227e-15
True
14 statistics: 12.853146132542978 pvalue: 5.997697174348817e-32
True
15 statistics: 6.849454815841208 pvalue: 3.652674361672202e-11
True
